In [8]:
import pandas as pd
import re
import os

# === CONFIGURAÇÕES DE CAMINHO ===
PASTA = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
ARQUIVO_ENTRADA = os.path.join(PASTA, "paragrafos_validos_corpus.xlsx")
ARQUIVO_SAIDA = os.path.join(PASTA, "paragrafos_filtrados_corpus.xlsx")
ARQUIVO_RELATORIO = os.path.join(PASTA, "relatorio_filtragem_paragrafos.xlsx")

# === FUNÇÃO DE FILTRAGEM ===
def paragrafo_relevante(texto):
    texto_limpo = texto.lower().strip()
    padroes_excluir = [
        r'nomes?[:\-]',
        r'e-mail|site:|www\.',
        r'cep:|quadra|srtv|endere',
        r'coordenação-geral|departamento',
        r'ministério da saúde',
        r'esta obra.*licença',
        r'(nome|cargo) do autor',
        r'saúde pública brasileira.*acesso',
        r'\b(anexo|apêndice|índice)\b',
        r'(autores?|versão|tiragem|edição).*202[0-9]',
        r'(sigla|significado) de ',
        r'(organizador|responsável|revisão)',
    ]
    for padrao in padroes_excluir:
        if re.search(padrao, texto_limpo):
            return False
    if len(texto_limpo.split()) < 6:
        return False
    return True

# === EXECUÇÃO ===
print("🔍 Lendo arquivo...")
df = pd.read_excel(ARQUIVO_ENTRADA)

print("⚙️ Aplicando filtro semântico...")
df["Relevante"] = df["Texto"].astype(str).apply(paragrafo_relevante)

df_filtrado = df[df["Relevante"]].drop(columns=["Relevante"])
df_excluido = df[~df["Relevante"]].drop(columns=["Relevante"])

print(f"✅ Parágrafos mantidos: {len(df_filtrado)}")
print(f"🚫 Parágrafos excluídos: {len(df_excluido)}")

# === SALVAR RESULTADO FILTRADO ===
print("💾 Salvando arquivo com parágrafos filtrados...")
df_filtrado.to_excel(ARQUIVO_SAIDA, index=False)

# === GERAR RELATÓRIO EM EXCEL COM AMOSTRAS ===
print("📝 Gerando relatório...")
resumo = {
    "Total original": [len(df)],
    "Total mantido": [len(df_filtrado)],
    "Total excluído": [len(df_excluido)],
    "% mantido": [round(100 * len(df_filtrado) / len(df), 2)],
    "% excluído": [round(100 * len(df_excluido) / len(df), 2)],
}
df_resumo = pd.DataFrame(resumo)

amostra_mantido = df_filtrado.sample(min(5, len(df_filtrado)), random_state=1)
amostra_excluido = df_excluido.sample(min(5, len(df_excluido)), random_state=1)

with pd.ExcelWriter(ARQUIVO_RELATORIO) as writer:
    df_resumo.to_excel(writer, sheet_name="Resumo", index=False)
    amostra_mantido.to_excel(writer, sheet_name="Amostra Mantido", index=False)
    amostra_excluido.to_excel(writer, sheet_name="Amostra Excluído", index=False)

print("📊 Relatório salvo com sucesso!")
print(f"📁 Arquivo filtrado: {ARQUIVO_SAIDA}")
print(f"📄 Relatório: {ARQUIVO_RELATORIO}")


🔍 Lendo arquivo...
⚙️ Aplicando filtro semântico...
✅ Parágrafos mantidos: 574153
🚫 Parágrafos excluídos: 50769
💾 Salvando arquivo com parágrafos filtrados...
📝 Gerando relatório...
📊 Relatório salvo com sucesso!
📁 Arquivo filtrado: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\paragrafos_filtrados_corpus.xlsx
📄 Relatório: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\relatorio_filtragem_paragrafos.xlsx


In [9]:
#diagnosticar_pasta Antes da Etapa 2 – Extração e limpeza de texto Diagnóstico visual de duplicados e nomes corrompidos

import os
import hashlib
import re
import csv
from pathlib import Path
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext

# === FUNÇÕES AUXILIARES ===

def calcular_hash(caminho_arquivo, limite=1024*1024):
    """Calcula hash de uma parte do arquivo (1 MB por padrão) para diagnóstico rápido."""
    sha256 = hashlib.sha256()
    with open(caminho_arquivo, 'rb') as f:
        sha256.update(f.read(limite))
    return sha256.hexdigest()

def normalizar_nome(nome):
    """Remove repetições consecutivas de sufixos."""
    padrao = r'(txt_\w+\d{4})+'
    return re.sub(padrao, lambda m: m.group(1), nome)

def diagnosticar_pasta(pasta_raiz, saida_texto):
    pasta_raiz = Path(pasta_raiz)
    hash_map = {}
    log_linhas = []
    log_path = pasta_raiz / "log_diagnostico.csv"

    saida_texto.insert(tk.END, f"🔍 Diagnóstico em: {pasta_raiz}\n")
    saida_texto.insert(tk.END, "Apenas leitura — nenhum arquivo será alterado.\n\n")
    saida_texto.update()

    for caminho_arquivo in pasta_raiz.rglob("*.txt"):
        hash_arquivo = calcular_hash(caminho_arquivo)
        nome_original = caminho_arquivo.name
        caminho_pasta = caminho_arquivo.parent

        # Verifica duplicidade de conteúdo
        if hash_arquivo in hash_map:
            saida_texto.insert(tk.END, f"⚠️ Possível duplicado: {nome_original} (igual a {hash_map[hash_arquivo].name})\n")
            log_linhas.append(["DUPLICADO", str(caminho_arquivo), "Mesma hash de", str(hash_map[hash_arquivo])])
        else:
            hash_map[hash_arquivo] = caminho_arquivo

        # Verifica nome corrompido
        nome_corrigido = normalizar_nome(nome_original)
        if nome_corrigido != nome_original:
            saida_texto.insert(tk.END, f"🔁 Nome suspeito: {nome_original} → Sugerido: {nome_corrigido}\n")
            log_linhas.append(["NOME_CORROMPIDO", str(caminho_arquivo), "Sugerido", nome_corrigido])

    with open(log_path, mode='w', newline='', encoding='utf-8') as log_csv:
        writer = csv.writer(log_csv)
        writer.writerow(["Tipo", "Arquivo", "Info", "Referência/Sugestão"])
        writer.writerows(log_linhas)

    saida_texto.insert(tk.END, f"\n📄 Log salvo em: {log_path}\n")
    saida_texto.insert(tk.END, "✅ Diagnóstico concluído.\n")
    saida_texto.update()

# === INTERFACE GRÁFICA ===

def iniciar_interface():
    def escolher_pasta():
        pasta = filedialog.askdirectory(title="Selecione a pasta 'corpus_final'")
        if pasta:
            entrada_pasta.delete(0, tk.END)
            entrada_pasta.insert(0, pasta)

    def executar_diagnostico():
        pasta_escolhida = entrada_pasta.get()
        if not os.path.isdir(pasta_escolhida):
            messagebox.showerror("Erro", "Selecione uma pasta válida.")
            return
        saida_texto.delete(1.0, tk.END)
        diagnosticar_pasta(pasta_escolhida, saida_texto)

    janela = tk.Tk()
    janela.title("🔎 Diagnóstico de Arquivos Duplicados e Nomes Corrompidos")

    tk.Label(janela, text="📁 Pasta Base:").pack(pady=5)
    entrada_pasta = tk.Entry(janela, width=60)
    entrada_pasta.pack()
    tk.Button(janela, text="Selecionar Pasta", command=escolher_pasta).pack(pady=5)

    tk.Button(janela, text="🔍 Executar Diagnóstico", command=executar_diagnostico, bg="blue", fg="white").pack(pady=10)

    saida_texto = scrolledtext.ScrolledText(janela, width=100, height=25)
    saida_texto.pack(padx=10, pady=10)

    janela.mainloop()

# Rodar
if __name__ == "__main__":
    iniciar_interface()


In [10]:
#executar_log_diagnostico remove duplicatas reais e renomeia arquivos com nome corrompido.Logo após o diagnóstico, ainda na Etapa 2
import os
import csv
from pathlib import Path

# === CONFIGURAÇÃO ===
PASTA_BASE = Path(r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final")  # ajuste se necessário
CAMINHO_LOG_DIAGNOSTICO = PASTA_BASE / "log_diagnostico.csv"
CAMINHO_LOG_EXECUCAO = PASTA_BASE / "log_execucao.csv"

# === LEITURA DO LOG ===
with open(CAMINHO_LOG_DIAGNOSTICO, encoding='utf-8') as f:
    leitor = csv.DictReader(f)
    acoes = list(leitor)

# === EXECUÇÃO DAS AÇÕES ===
log_execucao = []

for linha in acoes:
    tipo = linha['Tipo']
    caminho_str = linha['Arquivo']
    caminho_arquivo = Path(caminho_str)

    if not caminho_arquivo.exists():
        log_execucao.append(["IGNORADO", caminho_str, "Arquivo não encontrado"])
        continue

    if tipo == "DUPLICADO":
        try:
            caminho_arquivo.unlink()
            log_execucao.append(["REMOVIDO", caminho_str, "Duplicata removida"])
            print(f"🗑️ Removido: {caminho_str}")
        except Exception as e:
            log_execucao.append(["ERRO", caminho_str, f"Erro ao remover: {e}"])

    elif tipo == "NOME_CORROMPIDO":
        sugestao = linha['Referência/Sugestão']
        novo_caminho = caminho_arquivo.parent / sugestao
        contador = 1
        while novo_caminho.exists():
            novo_caminho = caminho_arquivo.parent / f"{sugestao}_{contador}.txt"
            contador += 1
        try:
            caminho_arquivo.rename(novo_caminho)
            log_execucao.append(["RENOMEADO", caminho_str, f"Novo nome: {novo_caminho.name}"])
            print(f"✏️ Renomeado: {caminho_arquivo.name} → {novo_caminho.name}")
        except Exception as e:
            log_execucao.append(["ERRO", caminho_str, f"Erro ao renomear: {e}"])

# === SALVA LOG FINAL ===
with open(CAMINHO_LOG_EXECUCAO, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Ação", "Arquivo", "Detalhes"])
    writer.writerows(log_execucao)

print(f"\n✅ Execução concluída. Log salvo em: {CAMINHO_LOG_EXECUCAO}")


✅ Execução concluída. Log salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\log_execucao.csv


In [11]:
#filtro_paragrafos_relevantes Filtra semanticamente os parágrafos
import pandas as pd
import os
import re
from collections import Counter
from datetime import datetime

# === CAMINHOS ===
PASTA_CORPUS = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
ARQUIVO_PARAGRAFOS = os.path.join(PASTA_CORPUS, "paragrafos_validos_corpus.xlsx")
ARQUIVO_SAIDA = os.path.join(PASTA_CORPUS, f"perguntas_geradas_{datetime.today().strftime('%Y%m%d')}.xlsx")

# === FUNÇÃO PARA GERAR PERGUNTAS COMUNS ===
def gerar_perguntas(texto):
    perguntas = []

    # Baixo nível de NLP, apenas heurístico (foco em utilidade prática)
    if re.search(r"\bprocedimento[s]?\b", texto, re.IGNORECASE):
        perguntas.append("Quais são os procedimentos recomendados?")
    if re.search(r"\bexame[s]?\b", texto, re.IGNORECASE):
        perguntas.append("Como é realizado o exame?")
    if re.search(r"\btratamento\b", texto, re.IGNORECASE):
        perguntas.append("Qual é o tratamento indicado?")
    if re.search(r"\bnotifica(ção|r)\b", texto, re.IGNORECASE):
        perguntas.append("Quando o caso deve ser notificado?")
    if re.search(r"\bmedicamento[s]?\b", texto, re.IGNORECASE):
        perguntas.append("Quais medicamentos devem ser utilizados?")
    if re.search(r"\bsintoma[s]?\b", texto, re.IGNORECASE):
        perguntas.append("Quais são os sintomas?")
    if re.search(r"\bmonitoramento\b", texto, re.IGNORECASE):
        perguntas.append("Como é feito o monitoramento?")
    if re.search(r"\bvigilância\b", texto, re.IGNORECASE):
        perguntas.append("Quais são as ações de vigilância indicadas?")
    if re.search(r"\bsurto[s]?\b", texto, re.IGNORECASE):
        perguntas.append("O que deve ser feito em caso de surto?")

    return perguntas

# === CARGA DE PARÁGRAFOS ===
df = pd.read_excel(ARQUIVO_PARAGRAFOS)

# === GERAR PERGUNTAS POR PARÁGRAFO ===
linhas_saida = []

for _, row in df.iterrows():
    arquivo = row["Arquivo"]
    texto = str(row["Texto"])
    perguntas = gerar_perguntas(texto)

    for pergunta in perguntas:
        linhas_saida.append({
            "Arquivo (Tema)": arquivo,
            "Pergunta sugerida": pergunta,
            "Texto de origem": texto
        })

# === SALVAR RESULTADO ===
df_resultado = pd.DataFrame(linhas_saida)
df_resultado.to_excel(ARQUIVO_SAIDA, index=False)

print("✅ Perguntas geradas com sucesso!")
print(f"📄 Arquivo salvo em: {ARQUIVO_SAIDA}")


✅ Perguntas geradas com sucesso!
📄 Arquivo salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_geradas_20250802.xlsx


In [12]:
# === TESTE COM AMOSTRA DE 100 PARÁGRAFOS ===
df = pd.read_excel(ARQUIVO_PARAGRAFOS).sample(100, random_state=42)

In [13]:
print(df.head(100))

                                                  Arquivo  Parágrafo nº  \
346681  metapneumovirus_guia_1txt_guiavsa2023txt_guiav...          1576   
516470                        tuberculose_protocolo_3.txt         17520   
48466   coqueluche_nota_tecnica_1txt_notatecnica2023tx...           285   
216636                         hepatite_c_protocolo_1.txt          2705   
617515  zoonoses_manual_1txt_manual2023txt_manual2023t...           360   
...                                                   ...           ...   
615593  zoonoses_manual_1txt_manual2023txt_manual2023.txt          2779   
459494                 sifilis_manual_1txt_manual2023.txt           697   
555755  tuberculose_protocolo_6txt_protocolo2023txt_pr...         10111   
235798  hepatite_e_protocolo_1txt_protocolo2023txt_pro...          2145   
388526  peste_manual_1txt_manual2023txt_manual2023txt_...          1788   

                                                    Texto  
346681  dos sintomas respiratórios, sem

In [14]:
#gerador_perguntas_t5
# === CONTINUAR AQUI COM O MESMO SCRIPT ===
linhas = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Gerando perguntas - Amostra"):
    texto = str(row.get("Texto", "")).strip()
    if not texto:
        continue
    try:
        perguntas = gerar_perguntas_t5(texto)
        for pergunta in perguntas:
            linhas.append({
                "Arquivo (Tema)": row.get("Arquivo", ""),
                "Pergunta sugerida": pergunta,
                "Texto de origem": texto
            })
    except Exception as e:
        print(f"⚠️ Erro na linha {idx}: {e}")

# === SALVAR RESULTADO FINAL ===
ARQUIVO_TESTE = os.path.join(PASTA_CORPUS, "perguntas_amostra.xlsx")
df_resultado = pd.DataFrame(linhas)
df_resultado.to_excel(ARQUIVO_TESTE, index=False)

print("✅ Perguntas da amostra geradas com sucesso!")
print(f"📄 Arquivo salvo em: {ARQUIVO_TESTE}")


Gerando perguntas - Amostra:   0%|          | 0/100 [00:00<?, ?it/s]c:\Users\isisi\anaconda3\Lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Gerando perguntas - Amostra: 100%|██████████| 100/100 [01:48<00:00,  1.08s/it]

✅ Perguntas da amostra geradas com sucesso!
📄 Arquivo salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final\perguntas_amostra.xlsx


In [2]:
# Instalação dos pacotes necessários (pode deixar só na primeira célula do notebook)
%pip install transformers torch pandas openpyxl tqdm sentencepiece

# === IMPORTAÇÕES ===
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datetime import datetime
from tqdm import tqdm
import os

# === CONFIGURAÇÃO DE CAMINHOS ===
PASTA_CORPUS = r"C:\Users\isisi\Documents\IAVS_PROJETO\corpus_final"
ARQUIVO_PARAGRAFOS = os.path.join(PASTA_CORPUS, "paragrafos_validos_corpus.xlsx")
ARQUIVO_SAIDA = os.path.join(PASTA_CORPUS, f"perguntas_avancadas_{datetime.today().strftime('%Y%m%d')}.xlsx")

# === CARREGAR MODELO E TOKENIZER ===
MODELO = "valhalla/t5-base-qg-hl"
tokenizer = T5Tokenizer.from_pretrained(MODELO)
model = T5ForConditionalGeneration.from_pretrained(MODELO)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# === FUNÇÃO DE GERAÇÃO DE PERGUNTAS ===
def gerar_perguntas_t5(texto):
    entrada = f"generate question: {texto} </s>"
    input_ids = tokenizer.encode(entrada, return_tensors="pt").to(device)
    with torch.no_grad():  # otimiza uso de memória
        saida_ids = model.generate(
            input_ids=input_ids,
            max_length=64,
            num_beams=4,
            num_return_sequences=3,
            early_stopping=True
        )
    perguntas = [tokenizer.decode(g, skip_special_tokens=True) for g in saida_ids]
    return list(set(perguntas))  # remove duplicadas

# === CARREGAR PARÁGRAFOS ===
df = pd.read_excel(ARQUIVO_PARAGRAFOS)
linhas = []

# === PERCORRER PARÁGRAFOS COM BARRA DE PROGRESSO ===
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Gerando perguntas"):
    texto = str(row.get("Texto", "")).strip()
    if not texto:
        continue
    try:
        perguntas = gerar_perguntas_t5(texto)
        for pergunta in perguntas:
            linhas.append({
                "Arquivo (Tema)": row.get("Arquivo", ""),
                "Pergunta sugerida": pergunta,
                "Texto de origem": texto
            })
        # Checkpoint: salva a cada 100 parágrafos
        if idx % 100 == 0 and idx != 0:
            pd.DataFrame(linhas).to_excel(ARQUIVO_SAIDA, index=False)
    except Exception as e:
        print(f"⚠️ Erro na linha {idx}: {e}")

# === SALVAR RESULTADO FINAL ===
df_resultado = pd.DataFrame(linhas)
df_resultado.to_excel(ARQUIVO_SAIDA, index=False)

print("✅ Perguntas geradas com sucesso!")
print(f"📄 Arquivo salvo em: {ARQUIVO_SAIDA}")

Note: you may need to restart the kernel to use updated packages.


Gerando perguntas:   0%|          | 0/624922 [00:00<?, ?it/s]c:\Users\isisi\anaconda3\Lib\site-packages\transformers\models\t5\tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Gerando perguntas:   1%|          | 3201/624922 [1:21:07<153:02:08,  1.13it/s]

⚠️ Erro na linha 3200: [Errno 13] Permission denied: 'C:\\Users\\isisi\\Documents\\IAVS_PROJETO\\corpus_final\\perguntas_avancadas_20250802.xlsx'


Gerando perguntas:   4%|▍         | 24604/624922 [9:43:12<237:09:42,  1.42s/it]


KeyboardInterrupt: 